In [10]:
import requests
import pandas as pd
import io

In [3]:
bond_url = 'https://iss.moex.com/iss/apps/infogrid/emission/rates.csv?iss.dp=comma&iss.df=%25d.%25m.%25Y&iss.tf=%25H:%25M:%25S&iss.dtf=%25d.%25m.%25Y%25H:%25M:%25S&iss.only=rates&limit=unlimited&lang=ru'

In [4]:
req = requests.get(bond_url).content

In [16]:
raw_data = pd.read_csv(io.StringIO(req.decode('cp1251')), sep=';', header= 1)
raw_data

,SECID,SHORTNAME,NAME,TYPENAME,ISIN,REGNUMBER,LISTLEVEL,FACEVALUE,FACEUNIT,ISSUESIZE,...,RTL3,RTH3,DISCOUNT1,LIMIT1,DISCOUNT2,LIMIT2,DISCOUNT3,DISCOUNTL0,DISCOUNTH0,FULLCOVERED
0,AMUNIBB2AER2,UBANK11/22,UNIBANK OJSC Series 1,Корпоративные еврооблигации,AMUNIBB2AER2,NaN,3.0,"100,0",USD,10000.0,...,"0,0","12094,46","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
1,AMUNIBB2DER6,UBANK02/24,UNIBANK OJSC Series 1 21,Корпоративные еврооблигации,AMUNIBB2DER6,NaN,3.0,"100,0",USD,33478.0,...,"0,0","12205,88","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2,AT0000A2UF10,Raiff CAG,Raiffeisen Centrobank AG,Корпоративные еврооблигации,AT0000A2UF10,NaN,3.0,"1000,0",USD,50000.0,...,"0,0","100656,56","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
3,CH0248531110,VTB-24 CHF,VTB CAPITAL S.A. 24 CHF,Корпоративные еврооблигации,CH0248531110,NaN,NaN,"5000,0",CHF,70000.0,...,"0,0","664131,56","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
4,CH0379268748,RZD-23 CHF,RZD Capital PLC 23 CHF,Корпоративные еврооблигации,CH0379268748,NaN,NaN,"5000,0",CHF,90000.0,...,"0,0","404635,46","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,XS2420560869,BCS01/25,BCS SP Plc Series 239,Корпоративные еврооблигации,XS2420560869,NaN,3.0,"100000,0",RUB,700.0,...,"0,0","200000,0","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2657,XS2423361190,BCS02/27-4,BCS SP Plc Series 240,Корпоративные еврооблигации,XS2423361190,NaN,3.0,"1250,0",USD,8000.0,...,"0,0","150638,06","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2658,XS2429208486,BCS05/25,BCS SP Plc Series 243,Корпоративные еврооблигации,XS2429208486,NaN,3.0,"1250,0",USD,8000.0,...,"0,0","139631,56","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0
2659,XS2439218640,BCS06/25-B,BCS SP Plc Series 244,Корпоративные еврооблигации,XS2439218640,NaN,3.0,"1250,0",USD,8000.0,...,"0,0","147079,8","100,0",NaN,NaN,NaN,NaN,"100,00000","100,00000",1.0


In [17]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2661 entries, 0 to 2660
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SECID                   2661 non-null   object 
 1   SHORTNAME               2661 non-null   object 
 2   NAME                    2661 non-null   object 
 3   TYPENAME                2661 non-null   object 
 4   ISIN                    2661 non-null   object 
 5   REGNUMBER               2276 non-null   object 
 6   LISTLEVEL               2395 non-null   float64
 7   FACEVALUE               2661 non-null   object 
 8   FACEUNIT                2661 non-null   object 
 9   ISSUESIZE               2644 non-null   float64
 10  IS_COLLATERAL           2661 non-null   int64  
 11  IS_EXTERNAL             2661 non-null   int64  
 12  PRIMARY_BOARDID         2661 non-null   object 
 13  PRIMARY_BOARD_TITLE     2661 non-null   object 
 14  MATDATE                 2609 non-null   